In [1]:
import rasterio as rio
from rasterio import features
import geopandas as gpd
import time
from matplotlib import pyplot as plt
import numpy as np
from helpers import spatial_agg_point_df
from shapely.geometry import Point
from fiona.crs import from_epsg
from glob import glob
import os
import pickle

PyTables is not installed. No support for HDF output.


# use a function to do it

In [2]:
from helpers import create_global_agg_var_grid, create_global_agg_CONF_grid


In [ ]:
with open('climate_grid_meta_0_25_wgs84.pickle', 'rb') as fp:
    meta1 = pickle.load(fp)
    
mf = 'climate_grid_meta_0_25_wgs84.pickle'
fire_pts_folders = !ls -d ../modis_fire_points/*
shp_files = [glob(folder + '/*.shp')[0] for folder in fire_pts_folders]

for ag in [2.5, 0.25]:    
    
    try:
        out_dir = f'D:\\projects\\RD\\night_fire\\vars\grid_{ag}_degree_vars'.replace('.','_')
        create_global_agg_var_grid(shp_files, meta_file=mf, agg=ag, conf=10, out_folder=out_dir)
    
    except Exception as e:
        print('2019 does not have a TYPE field...')

In [5]:
with open('climate_grid_meta_0_25_wgs84.pickle', 'rb') as fp:
    meta1 = pickle.load(fp)

mf = 'climate_grid_meta_0_25_wgs84.pickle'
fire_pts_folders = !ls -d ../modis_fire_points/*
shp_files = [glob(folder + '/*.shp')[0] for folder in fire_pts_folders]

for ag in [2.5, 0.25]:

    try:
        out_dir = f'D:\\projects\\RD\\night_fire\\vars\grid_{ag}_degree_CONF'.replace('.','_')
        create_global_agg_CONF_grid(shp_files, meta_file=mf, agg=ag, conf=10, out_folder=out_dir)
    
    except Exception as e:
        print('2019 does not have TYPE field...')

on year 2000
writing out November CONF rasters
writing out December CONF rasters
on year 2001
writing out January CONF rasters
writing out February CONF rasters
writing out March CONF rasters
writing out April CONF rasters
writing out May CONF rasters
writing out June CONF rasters
writing out July CONF rasters
writing out August CONF rasters
writing out September CONF rasters
writing out October CONF rasters
writing out November CONF rasters
writing out December CONF rasters
on year 2002
writing out January CONF rasters
writing out February CONF rasters
writing out March CONF rasters
writing out April CONF rasters
writing out May CONF rasters
writing out June CONF rasters
writing out July CONF rasters
writing out August CONF rasters
writing out September CONF rasters
writing out October CONF rasters
writing out November CONF rasters
writing out December CONF rasters
on year 2003
writing out January CONF rasters
writing out February CONF rasters
writing out March CONF rasters
writing ou

In [ ]:
help(create_global_agg_var_grid)